# Ariel ML Challenge Baseline

Notebook presenting the baseline model for the [Ariel ML challenge 2021](https://www.ariel-datachallenge.space/).

In [ ]:
# General imports
import os
import numpy as np
import matplotlib.pylab as plt
import torch

from pathlib import Path

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Data access

In [ ]:
lc_train_path = ...  # link to training light curves directory
params_train_path = ...  # linl to training parameters directory
lc_test_path = ...  # link to evaluation light curves directory

n_wavelengths = 55
n_timesteps = 300

Let's define a Dataset using Pytorch utility

In [ ]:
from torch.utils.data import Dataset

class ArielMLDataset(Dataset):
    def __init__(self, lc_path, params_path=None, transform=None, start_ind=0, 
                 max_size = int(1e9), shuffle=True, seed=None, device=None):
        """Dataset to read files for the Ariel ML Data challenge 2021
        
        Args:
            lc_path: str
                path to the folder containing the light curves files
            params_path: str
                path to the folder containing the target transit depths (optional)
            transform: callable
                transformation to apply to the input light curves
            start_ind: int
                where to start reading the files from (after ordering)
            max_size: int
                maximum dataset size
            shuffle: bool
                whether to shuffle the dataset order or not
            seed: int
                numpy seed to set in case of shuffling
            device: str
                torch device
        """
        self.lc_path = lc_path
        self.transform = transform
        self.device = device

        self.files = sorted([p for p in os.listdir(self.lc_path) if p.endswith('txt')])
        if shuffle:
            np.random.seed(seed)
            np.random.shuffle(self.files)
        self.files = self.files[start_ind:start_ind+max_size]

        if params_path is not None:                
            self.params_path = params_path
        else:
            self.params_path = None
            self.params_files = None

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        item_lc_path = Path(self.lc_path) / self.files[idx]
        lc = torch.from_numpy(np.loadtxt(item_lc_path))
        if self.transform:
            lc = self.transform(lc)
        if self.params_path is not None:
            item_params_path = Path(self.params_path) / self.files[idx]
            target = torch.from_numpy(np.loadtxt(item_params_path))
        else:
            target = torch.Tensor()
        return {'lc': lc.to(self.device), 
                'target': target.to(self.device)}


### let's plot a random spectral light curve

In [ ]:
dataset = ArielMLDataset(lc_train_path, params_train_path, shuffle=True)

idx = np.random.randint(len(dataset))
item = dataset[idx]
offsets = np.linspace(-0.05, 0.05, item['lc'].shape[0])
f, ax = plt.subplots(figsize=(13,9))
plt.plot(item['lc'].T.detach().numpy() + offsets , label=None)
ax.legend([round(x, 4) for x in item['target'].detach().numpy()], fontsize=6, loc='lower right')


## Define simples preprocessing steps
- smoothing 
- clipping
- normalisation per wavelength
- removing ramp?

In [ ]:
def transform(x):
    """Perform a simple preprocessing of the input light curve array
    Args:
        x: np.array
            first dimension is time, at least 30 timesteps
    Return:
        preprocessed array
    """
    out = x.clone()
    # naive croping of main ramp part (replaced by 1.)
    out[:30] = 1.
    # centering
    out -=  1.
    # rough rescaling 
    out /= 0.04
    return out


Let's include these steps in the datasets for convenience

In [ ]:
train_size = 64  #4 * 128
val_size = 64  #4 * 128
test_size = 16  #1024

# Training
dataset_train = ArielMLDataset(lc_train_path, params_train_path, shuffle=True, start_ind=0, 
                               max_size=train_size, transform=transform)
# Validation
dataset_val = ArielMLDataset(lc_train_path, params_train_path, shuffle=True, start_ind=train_size, 
                             max_size=val_size, transform=transform)

# Testing
dataset_test = ArielMLDataset(lc_train_path, params_train_path, start_ind=train_size+val_size, 
                              shuffle=True, max_size=test_size, transform=transform)

# Evaluation : no output path available here, this will only be used for submission
dataset_eval = ArielMLDataset(lc_test_path, shuffle=True, transform=transform)

Let's define the corresponding data loaders, still using Pytorch utils module

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size = int(train_size / 4)

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=batch_size)
loader_test = DataLoader(dataset_test, batch_size=batch_size)
loader_eval = DataLoader(dataset_eval, batch_size=batch_size)

### Challenge Metric

The scoring system used for evaluation is defined here: https://www.ariel-datachallenge.space/ML/documentation/scoring

Let's define it here, with unity weights as we don't have the actual weights available.

In [ ]:
class ChallengeMetric:
    def __init__(self, weights=None):
        self.weights = weights
        
    def __call__(self, y, pred):
        y = y
        pred = pred
        if self.weights is None:
            weights = torch.ones_like(y, requires_grad=False)
        else:
            weights = self.weights

        return (weights * y * torch.abs(pred - y)).sum() / weights.sum() * 1e6
    
    def score(self, y, pred):
        y = y
        pred = pred
        if self.weights is None:
            weights = torch.ones_like(y, requires_grad=False)
        else:
            weights = self.weights

        return (1e4 - 2 * (weights * y * torch.abs(pred - y)).sum() / weights.sum() * 1e6)  
    
challenge_metric = ChallengeMetric()

## Models

A constant prediction model for reference

In [ ]:
naive_1 = lambda x: torch.ones(x.shape[:-1]) * 0.06  

The baseline model, a fully connected neural network with 2 hidden layers with ReLU activation functions.

In [ ]:
from torch.nn import Module, Sequential

class Baseline(Module):
    def __init__(self, H1=1024, H2=256, input_dim=n_wavelengths*n_timesteps, output_dim=n_wavelengths):
        super().__init__()
        self.network = Sequential(torch.nn.Linear(input_dim, H1),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(H1, H2),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(H2, output_dim),
                                   )
        
    def __call__(self, x):
        out = torch.flatten(x, start_dim=1)  # Need to flatten out the input light curves for this type network
        out = self.network(out)
        return out
    
baseline = Baseline().double().to(device)

### Training the baseline

In [ ]:
from torch.optim import Adam
from torch.nn import MSELoss, L1Loss

opt = Adam(baseline.parameters())
loss_function = MSELoss()
# loss_function = ChallengeMetric()
# loss_function = L1Loss()

challenge_metric = ChallengeMetric()
train_losses = []
val_losses = []
val_scores = []

In [ ]:
epochs = 1


for epoch in range(epochs):
    print(epoch)
    train_loss = 0
    val_loss = 0
    val_score = 0
    for k, item in enumerate(loader_train):
        pred = baseline(item['lc'])
        loss = loss_function(item['target'], pred)
        opt.zero_grad()
        loss.backward()
        opt.step()    
        train_loss += loss.item()
    train_loss = train_loss / len(loader_train)
    for k, item in enumerate(loader_val):
        pred = baseline(item['lc'])
        loss = loss_function(item['target'], pred)
        score = challenge_metric.score(item['target'], pred)
        val_loss += loss.item()
        val_score += score.item()
    val_loss /= len(loader_val)
    val_score /= len(loader_val)
    print('Training loss', round(train_loss, 6))
    print('Val loss', round(val_loss, 6))
    print('Val score', round(val_score, 2))
    
    train_losses += [train_loss]
    val_losses += [val_loss]
    val_scores += [val_score]

Let's look at the learning curve

In [ ]:
plt.plot(train_losses, '-o', label='Train Loss')
plt.plot(val_losses, '-o', label='Val Loss')
plt.xlabel('epochs')
plt.ylabel(loss_function)
plt.yscale('log')
plt.show()
plt.plot(val_scores, '-o', label='Val Score')
plt.xlabel('epochs')
plt.ylabel('Challenge score (unity weights)')
# plt.yscale('log')
plt.ylim(5000,10000)
plt.show()

## Compare models

In [ ]:
item = next(iter(loader_test))

preds = {'naive1': naive_1(item['lc']), 
         'normal_1000ppm': torch.normal(item['target'], 1e-3),
         'baseline': baseline(item['lc'])
        }

for name, pred in preds.items():
    print(name, f"\t{challenge_metric(item['target'], pred).item():.2f}")

### Produce evaluation vectors
(takes a few mins to run)

In [ ]:
%%time
import tqdm
preds = []
for k, item in tqdm.tqdm(enumerate(loader_eval)):
    preds += [baseline(item['lc'])]

eval_pred = torch.cat(preds).detach().numpy()

Let's quickly plot the mean results per wavelength

In [ ]:
plt.plot(eval_pred.mean(0), '-o')
plt.xlabel('wavelength')
plt.ylabel('mean prediction per wavelength')

And finally save the results as a txt file:

In [ ]:
save_path = './baseline_evaluation.txt'
if save_path and (53900, 55) == eval_pred.shape:
    np.savetxt(save_path, eval_pred)